# Plots and figures HCP

This notebook contains all code to generate the figures in the associated publications.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import os.path as path
import os
import socket
import sys

In [ ]:
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
print("Your style sheets are located at: {}".format(path.join(mpl.__path__[0], 'mpl-data', 'stylelib')))

import pandas as pd
import seaborn as sns

%matplotlib inline

In [ ]:
sys.path.append('..')

from configs.configs import get_config_dict
from helpers.figures import set_size

In [ ]:
plt.style.available

## Set configurations (shared across all figures)

In [ ]:
data_dimensionality = 'd15'
data_split = 'all'
kernel_param = 'kernel_lengthscales'

n_time_series = int(data_dimensionality[1:])

In [ ]:
hcp_cfg = get_config_dict(
    data_set_name='HCP_PTN1200_recon2',
    subset_dimensionality=data_dimensionality,
    hostname=socket.gethostname()
)

## TVFC estimates

### Fig. 6 - TVFC estimates (run on Hivemind)

In [ ]:
from benchmarks.fmri.rs.HCP_PTN1200_recon2.plotters.plot_TVFC_estimates import plot_tvfc_estimates

### Fig. 7 - TVFC estimates summary measures

In [ ]:
from benchmarks.fmri.rs.HCP_PTN1200_recon2.plotters.plot_TVFC_estimates_summary_measures import plot_tvfc_summary_measures_mean_over_subjects_all_edges

In [ ]:
model_name = 'SVWP_joint'
tvfc_summary_measure = 'mean'

tvfc_estimates_git_savedir = os.path.join(
    hcp_cfg['git-results-basedir'], 'TVFC_estimates', 'scan_0',
    data_split, 'multivariate', 'correlation'
)
file_name = f"{model_name:s}_TVFC_{tvfc_summary_measure:s}_mean_over_subjects.csv"
filepath = os.path.join(tvfc_estimates_git_savedir, file_name)
mean_over_subjects_tvfc_summaries_df = pd.read_csv(
    filepath,
    index_col=0
)  # (D, D)


plot_tvfc_summary_measures_mean_over_subjects_all_edges(
    config_dict=hcp_cfg,
    summarized_tvfc_df=mean_over_subjects_tvfc_summaries_df,
    summary_measure=tvfc_summary_measure,
    model_name=model_name,
    data_dimensionality='d15',
    figures_savedir='/Users/onnokampman/Developer/',
)

In [ ]:
from benchmarks.fmri.rs.HCP_PTN1200_recon2.plotters.plot_TVFC_estimates_summary_measures import plot_tvfc_summary_measures_mean_over_subjects_all_edges_joint

In [ ]:
plot_tvfc_summary_measures_mean_over_subjects_all_edges_joint(
    config_dict=hcp_cfg,
    summarized_tvfc_df=mean_over_subjects_tvfc_summaries_df,
    data_dimensionality='d15',
    # figures_savedir='/Users/onnokampman/Developer/',
)

## Subject measure prediction

### Fig. 8 - Morphometricity scores plot

In [ ]:
from benchmarks.fmri.rs.HCP_PTN1200_recon2.subject_measure_prediction.plot_morphometricity_scores_TVFC_summary_measures import plot_morphometricity_scores_joint

In [ ]:
subject_measures_subset = "cognitive"

subject_measures_list = hcp_cfg[f"subject-measures-{subject_measures_subset:s}"]
if subject_measures_subset == 'cognitive':
    subject_measures_list = hcp_cfg['subject-measures-nuisance-variables'] + subject_measures_list

scores_savedir = os.path.join(
    hcp_cfg['git-results-basedir'], 'subject_measure_prediction', 
    'morphometricity', subject_measures_subset
)


plot_morphometricity_scores_joint(
    hcp_cfg,
    scores_savedir,
    subject_measures_list,
    figures_savedir='/Users/onnokampman/Developer/'
)

### Fig. S - Out of sample plot

In [ ]:
from benchmarks.fmri.rs.HCP_PTN1200_recon2.subject_measure_prediction.plot_out_of_sample_prediction_scores_TVFC_summary_measures import _plot_out_of_sample_prediction_scores, _plot_out_of_sample_prediction_scores_joint, rename_variables_for_plots, _rename_models_for_plots

In [ ]:
metric = 'correlation'
tvfc_summary_measure = 'mean'
tvfc_estimation_method = 'SVWP_joint'
subject_measures_subset = 'cognitive'

subject_measures_list = hcp_cfg[f"subject-measures-{subject_measures_subset:s}"]
if subject_measures_subset == 'cognitive':
    subject_measures_list = hcp_cfg['subject-measures-nuisance-variables'] + subject_measures_list

scores_savedir = os.path.join(
    hcp_cfg['git-results-basedir'], 'subject_measure_prediction',
    'out_of_sample_prediction', subject_measures_subset
)

out_of_sample_prediction_results_df = pd.read_csv(
    os.path.join(
        scores_savedir, f'{metric:s}_linear_ridge_model_prediction_accuracy_TVFC_{tvfc_summary_measure:s}_{tvfc_estimation_method:s}.csv'
    ),
    index_col=0
)  # (n_permutations, n_subjects)
# print('results')
# print(out_of_sample_prediction_results_df)

out_of_sample_prediction_results_df = out_of_sample_prediction_results_df.loc[:, subject_measures_list]
out_of_sample_prediction_results_df = rename_variables_for_plots(out_of_sample_prediction_results_df, axis=0)
# out_of_sample_prediction_results_df = _rename_models_for_plots(hcp_cfg, out_of_sample_prediction_results_df)


_plot_out_of_sample_prediction_scores(
    config_dict=hcp_cfg,
    out_of_sample_prediction_results_df=out_of_sample_prediction_results_df,
    tvfc_summary_measure=tvfc_summary_measure,
    # figures_savedir='/Users/onnokampman/Developer/'
)

In [ ]:
_plot_out_of_sample_prediction_scores_joint(
    config_dict=hcp_cfg,
    scores_savedir=scores_savedir,
    subject_measures_list=subject_measures_list,
    # performance_metric='prediction_accuracy',
    performance_metric='r2_scores',
    tvfc_estimation_methods=hcp_cfg['plot-models'],
    # figures_savedir='/Users/onnokampman/Developer/'
)

## Test-retest

### Fig. 9 - Edgewise ICC and I2C2 scores

In [ ]:
from benchmarks.fmri.rs.HCP_PTN1200_recon2.test_retest.plot_ICC_edgewise_matrices import plot_icc_scores_per_edge

In [ ]:
icc_scores_filepath = os.path.join(
    hcp_cfg['git-results-basedir'], 'test_retest', 'correlation', f'{tvfc_summary_measure:s}_ICCs_{model_name:s}.csv'
)
summary_measure_icc_df = pd.read_csv(
    icc_scores_filepath, 
    index_col=0,
)


plot_icc_scores_per_edge(
    config_dict=hcp_cfg,
    icc_edgewise_df=summary_measure_icc_df,
    data_dimensionality='d15',
    tvfc_summary_measure=tvfc_summary_measure,
    model_name=model_name,
    # figures_savedir="/Users/onnokampman/Developer/",
)

In [ ]:
from benchmarks.fmri.rs.HCP_PTN1200_recon2.test_retest.plot_ICC_edgewise_matrices import plot_icc_scores_per_edge_joint

In [ ]:
plot_icc_scores_per_edge_joint(
    config_dict=hcp_cfg,
    data_dimensionality='d15',
    figures_savedir="/Users/onnokampman/Developer/",
)

In [ ]:
from benchmarks.fmri.rs.HCP_PTN1200_recon2.test_retest.plot_I2C2_scores import plot_i2c2_bar

In [ ]:
plot_i2c2_bar(
    config_dict=hcp_cfg,
    figures_savedir="/Users/onnokampman/Developer/",
)

## Brain state analysis

### Fig. S8 - Brain state clusters

In [ ]:
from benchmarks.frmi.rs.HCP_PTN1200_recon2.brain_states.plot_brain_states import _plot_brain_state_cluster_centroids

In [ ]:
_plot_brain_state_cluster_centroids(
    config_dict=hcp_cfg,
    # model_name='SVWP_joint',
    # model_name='DCC_joint',
    model_name='SW_cross_validated',
    n_basis_states=3,
    data_dimensionality='d15',
    # figures_savedir="/Users/onnokampman/Developer/"
)

#### Fig. S - Brain state clustering inertias

In [ ]:
from benchmarks.fmri.rs.HCP_PTN1200_recon2.brain_states.plot_brain_states import _plot_brain_state_cluster_centroids

In [ ]:
hcp_inertias_df = pd.read_csv(
    os.path.join(hcp_cfg['git-results-basedir'], 'brain_states', 'correlation_inertias_SVWP_joint.csv'),
    index_col=0
)
hcp_inertias_df

In [ ]:
fig, ax = plt.subplots(
    figsize=set_size()
)
sns.lineplot(
    data=hcp_inertias_df,
    markers=True,
    ax=ax
)
ax.set_xticks(hcp_cfg['n-brain-states-list'])
ax.set_xlabel('number of clusters')
ax.set_ylabel('inertia')
ax.legend(
    title="scan",
)

In [ ]:
fig.savefig(
    '../figures/hcp/brain_states/inertias_SVWP_joint.pdf',
    format='pdf',
    bbox_inches='tight'
)

In [ ]:
model_name = 'SVWP_joint'
n_brain_states = 3

_plot_brain_state_cluster_centroids(
    config_dict=hcp_cfg,
    model_name=model_name,
    n_basis_states=n_brain_states,
    data_dimensionality=data_dimensionality
)

#### Fig. 10 - Brain state switch rate

In [ ]:
from benchmarks.fmri.rs.HCP_PTN1200_recon2.brain_states.plot_brain_state_switch_counts import plot_brain_state_switch_count

In [ ]:
plot_brain_state_switch_count(
    config_dict=hcp_cfg,
    n_basis_states=3,
    # figure_savedir="/Users/onnokampman/Developer/"
)

## Imputation benchmark

### Fig. 13 - Imputation benchmark results

In [ ]:
from benchmarks.fmri.rs.HCP_PTN1200_recon2.imputation_study.plot_LEOO_likelihoods import plot_likelihoods_raincloud

In [ ]:
experiment_dimensionality = 'multivariate'

all_likelihoods_df = pd.DataFrame()
for model_name in hcp_cfg['plot-models']:
    likelihoods_filename = f'LEOO_{experiment_dimensionality:s}_likelihoods_{model_name:s}.csv'
    test_likelihoods_savedir = os.path.join(hcp_cfg['git-results-basedir'], 'imputation_study')
    test_likelihoods_filepath = os.path.join(test_likelihoods_savedir, likelihoods_filename)
    if os.path.exists(test_likelihoods_filepath):
        likelihoods_df = pd.read_csv(
            test_likelihoods_filepath, index_col=0
        )  # (n_subjects, n_scans)
        likelihoods_array = likelihoods_df.values.reshape(-1)  # (n_subjects * n_scans, 1)

        model_name = model_name.replace('SVWP_joint', 'WP')
        model_name = model_name.replace('DCC_joint', 'DCC-J')
        model_name = model_name.replace('DCC_bivariate_loop', 'DCC-BL')
        model_name = model_name.replace('SW_cross_validated', 'SW-CV')
        model_name = model_name.replace('_', '-')

        all_likelihoods_df[model_name] = likelihoods_array


plot_likelihoods_raincloud(
    config_dict=hcp_cfg,
    all_likelihoods_df=all_likelihoods_df,
    data_dimensionality='d15',
    experiment_dimensionality='multivariate',
    figure_savedir="/Users/onnokampman/Developer"
)

### Fig. 14 - Edgewise imputation benchmark results

In [ ]:
from benchmarks.fmri.rs.HCP_PTN1200_recon2.imputation_study.plot_edgewise_LEOO_likelihoods import plot_edgewise_imputation_benchmark_scores

In [ ]:
test_likelihoods_savedir = os.path.join(hcp_cfg['git-results-basedir'], 'imputation_study')
likelihoods_filename = f'LEOO_{experiment_dimensionality:s}_likelihoods_SVWP_joint_edgewise.csv'
likelihoods_df = pd.read_csv(
    os.path.join(test_likelihoods_savedir, likelihoods_filename),
    index_col=0
)  # (D, D)


plot_edgewise_imputation_benchmark_scores(
    config_dict=hcp_cfg,
    edgewise_likelihoods=likelihoods_df,
    model_name='SVWP_joint',
    figure_savedir="/Users/onnokampman/Developer"
)

In [ ]:
from benchmarks.fmri.rs.HCP_PTN1200_recon2.imputation_study.plot_edgewise_LEOO_likelihoods import plot_edgewise_imputation_benchmark_scores_joint

In [ ]:
plot_edgewise_imputation_benchmark_scores_joint(
    config_dict=hcp_cfg,
    # figure_savedir="/Users/onnokampman/Developer"
)

## Kernel lengthscales and optimal window length

In [ ]:
from helpers.kernel_analysis import plot_lengthscale_window_length_relation

In [ ]:
kernel_params_savedir = os.path.join(
    hcp_cfg['git-results-basedir'], 'kernel_analysis'
)
kernel_params_df = pd.read_csv(
    os.path.join(kernel_params_savedir, f'{kernel_param:s}_kernel_params.csv'),
    index_col=0
)
print(kernel_params_df)
optimal_window_lengths_df = pd.read_csv(
    os.path.join(hcp_cfg['git-results-basedir'], 'optimal_window_lengths', data_split, 'optimal_window_lengths.csv'),
    index_col=0
)
print(optimal_window_lengths_df)

# Prepare data for plot.
assert kernel_params_df.shape == optimal_window_lengths_df.shape
kernel_params_array = kernel_params_df.values.reshape(-1, 1)
optimal_window_lengths_array = optimal_window_lengths_df.values.reshape(-1, 1)

In [ ]:
plot_lengthscale_window_length_relation(
    hcp_cfg,
    kernel_params_array,
    optimal_window_lengths_array,
    # figures_savedir="/Users/onnokampman/Developer/"
)

### The end